# Initial set up


In [14]:
#Using Google colab secrets to store keys.
from google.colab import userdata

import urllib.parse

import requests

import json

lingq_api_key=userdata.get('LINGQ_API_KEY');
headers = {
    'Authorization': f'Token {lingq_api_key}',
    'Content-Type': 'application/json'
}


# Helper Functions

In [15]:
def get_json_response (url):
  response = requests.get(url, headers=headers)
  return response.json()

def print_json (json_parsed):
  print (json.dumps(json_parsed, indent = 4))



# Languages


In [16]:
languages = get_json_response('https://www.lingq.com/api/v2/languages/')
languages_with_known_words = [item["code"] for item in languages if item["knownWords"] != 0]
print_json(languages_with_known_words)



[
    "pl",
    "en"
]


# Courses for Language

In [18]:
# Import necessary libraries
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Define the base folder in Google Drive where you want to save the files
base_folder_name = 'LingQ_Data'

# Create or find the base folder
base_folder = None
folder_list = drive.ListFile({'q': "title='" + base_folder_name + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
if len(folder_list) == 0:
    base_folder = drive.CreateFile({'title': base_folder_name, 'mimeType': 'application/vnd.google-apps.folder'})
    base_folder.Upload()
else:
    base_folder = folder_list[0]

# Iterate through languages_with_known_words
for language_code in languages_with_known_words:
    print(f'Fetching language {language_code}')
    courses_in_language = get_json_response(f'https://www.lingq.com/api/v2/{language_code}/collections/my/')

    # Create or find the language folder within the base folder
    language_folder_name = language_code
    language_folder = None
    folder_list = base_folder['children']
    for folder in folder_list:
        if folder['title'] == language_folder_name:
            language_folder = folder
            break

    if language_folder is None:
        language_folder = drive.CreateFile({'title': language_folder_name, 'parents': [{'id': base_folder['id']}], 'mimeType': 'application/vnd.google-apps.folder'})
        language_folder.Upload()

    print_json(courses_in_language)

    for course in courses_in_language["results"]:
        course_title = course["title"]
        course_id = course["id"]
        print(f'Fetching course {course_title} - {course_id}')
        lessons_in_course = get_json_response(f'https://www.lingq.com/api/v2/{language_code}/collections/{course_id}')

        # Create or find the course folder within the language folder
        course_folder_name = course_title
        course_folder = None
        folder_list = language_folder['children']
        for folder in folder_list:
            if folder['title'] == course_folder_name:
                course_folder = folder
                break

        if course_folder is None:
            course_folder = drive.CreateFile({'title': course_folder_name, 'parents': [{'id': language_folder['id']}], 'mimeType': 'application/vnd.google-apps.folder'})
            course_folder.Upload()

        print_json(lessons_in_course)

        for lesson in lessons_in_course["lessons"]:
            lesson_title = lesson["title"]
            lesson_url = lesson["url"]
            lesson_json = get_json_response(lesson_url)

            if "tokenizedText" in lesson_json:
                sentences = [sentence[0]["text"] for sentence in lesson_json["tokenizedText"] if not ("opentag" in sentence[0]["tokens"][0])]
                text = " ".join(sentences)

                # Check if the file already exists within the course folder
                existing_files = drive.ListFile({'q': f"title='{lesson_title}.txt' and '{course_folder['id']}' in parents and trashed=false"}).GetList()

                if not existing_files:
                    # Create and upload a text file within the course folder
                    lesson_file = drive.CreateFile({'title': f'{lesson_title}.txt', 'parents': [{'id': course_folder['id']}]})
                    lesson_file.Upload()
                else:
                    print(f'File {lesson_title}.txt already exists in {course_folder_name}')




Fetching language pl


KeyError: ignored